In [37]:
import numpy as np
import pandas as pd
import os
import json
import time
import datetime
import hashlib
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import sys
import pymysql
from openpyxl import load_workbook


class Doris:
    def __init__(self, query, db='ed'):
        self.query = query
        self.params = {
            'host': 'data-center-doris-hw.tanjingpaas.com',
            'user': 'u_lipeiwen',
            'password': 'Lpw123!@',
            'port': 9030,
            'database': db,
            'charset': 'utf8mb4'
        }

    def read_doris_with_retries(self, max_retries=3, delay=5):
        retries = 0
        while retries < max_retries:
            try:
                conn = pymysql.connect(
                    host=self.params['host'],
                    user=self.params['user'],
                    password=self.params['password'],
                    database=self.params['database'],
                    port=self.params['port'],
                    charset=self.params['charset']
                )
                return pd.read_sql(self.query, conn)
            except pymysql.MySQLError as e:
                print(f"Failed to connect. Attempt {retries + 1} of {max_retries}. Error: {e}")
                retries += 1
                time.sleep(delay)
            finally:
                if 'conn' in locals():
                    conn.close()
        raise Exception("Failed to connect after max retries.")


In [72]:
sql = f"""
select 
if(chase_seal_status = '追封',1,0) ban_flag
,IF(a.phone_country = a.register_country,1,0)pho_reg_flag
,left(a.account,5)left_5_act
,SUBSTRING_INDEX(a.register_ip,'.',2)ip_ab
,LEFT(a.register_time,13)reg_hour
,a.*
from telegram_det_robot_register  a
LEFT JOIN telegram_agg_robot_account_info b on a.account=b.account
LEFT JOIN telegram_det_account_forbidden_record c on b.robot_id = c.robot_id
where ((date(chase_seal_time) = '2024-06-20' and date(check_time) <= '2024-06-20' ) 
or (date(chase_seal_time) = '2024-06-20' and check_time is null)
or (chase_seal_time is null and date(check_time) = '2024-06-20'))
and (register_msg like '注册成功%' or  register_msg = '登录成功') and source = '仓库2'
AND a.phone_country = '海地'


"""

doris_instance = Doris(sql, db='ed')
df = doris_instance.read_doris_with_retries()
df.head()
# df.tail()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15920\620937939.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(self.query, conn)


,ban_flag,pho_reg_flag,left_5_act,ip_ab,reg_hour,account,source,is_success,phone_area_short,phone_country,...,hardware_perf_cat,register_msg,seal_status,seal_time,chase_seal_status,chase_seal_time,check_time,login_time,login_msg,exc_time
0,0,1,50932,190.102,2024-06-19 20,50932374722,仓库2,true,+509,海地,...,,注册成功,None,None,None,None,2024-06-20 17:45:42,None,None,2024-06-20 17:45:42
1,0,0,50941,176.172,2024-06-18 19,50941431986,仓库2,true,+509,海地,...,,注册成功,None,None,None,None,2024-06-20 17:44:23,None,None,2024-06-20 17:44:23
2,1,1,50935,200.113,2024-06-19 22,50935200299,仓库2,true,+509,海地,...,,注册成功,None,None,追封,2024-06-20 19:28:35,None,None,None,2024-06-20 19:28:35
3,0,0,50941,222.240,2024-05-31 18,50941109778,仓库2,true,+509,海地,...,,注册成功,None,None,None,None,2024-06-20 17:36:34,2024-06-09 17:01:31,authToken登录失败(verification_code),2024-06-20 17:36:34
4,1,1,50941,190.115,2024-06-19 22,50941855786,仓库2,true,+509,海地,...,,注册成功,None,None,追封,2024-06-20 19:20:21,None,None,None,2024-06-20 19:20:21


In [73]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import DataBarRule
import os

# 假设已经有 final_result DataFrame

# 将 DataFrame 写入 Excel 文件
excel_file = r"C:\Users\Administrator\Desktop\tg注册封号分析\result2.xlsx"

# 定义字段
fields_to_analyze = [
    'protocol_type', 'pho_reg_flag', 'left_5_act', 'reg_hour',
    'hardware_system_language', 'hardware_tzoffset', 'hardware_version',
    'hardware_app_version'
]

# 初始化一个空的列表用于存储每个字段的封号比例结果
result_frames = []

# 计算每个字段的封号比例
for field in fields_to_analyze:
    total_count = df.groupby(field)['ban_flag'].count()
    ban_count = df.groupby(field)['ban_flag'].sum()
    ban_rate = ban_count / total_count
    
    result = pd.DataFrame({
        '统计字段': total_count.index,
        '总数': total_count.values,
        '封号数': ban_count.values,
        '封号比例': ban_rate.values,
    })
    
    result['字段'] = field  # 添加一列标识当前字段
    result_frames.append(result)

# 垂直拼接所有结果
final_result = pd.concat(result_frames, ignore_index=True)

# 将 DataFrame 写入 Excel 文件
final_result.to_excel(excel_file, index=False)

# 加载 Excel 文件并选择工作表
wb = load_workbook(excel_file)
ws = wb.active

# 定义浅色底纹填充样式
fill_patterns = {
    'protocol_type': PatternFill(start_color="FFFACD", end_color="FFFACD", fill_type="solid"),  # Lemon Chiffon
    'pho_reg_flag': PatternFill(start_color="FFE4E1", end_color="FFE4E1", fill_type="solid"),  # Misty Rose
    'left_5_act': PatternFill(start_color="B0E0E6", end_color="B0E0E6", fill_type="solid"),  # Powder Blue
    'reg_hour': PatternFill(start_color="F0FFF0", end_color="F0FFF0", fill_type="solid"),  # Honeydew
    'hardware_system_language': PatternFill(start_color="FFDAB9", end_color="FFDAB9", fill_type="solid"),  # Peach Puff
    'hardware_tzoffset': PatternFill(start_color="F0FFFF", end_color="F0FFFF", fill_type="solid"),  # Azure
    'hardware_version': PatternFill(start_color="87CEFA", end_color="87CEFA", fill_type="solid"),  # Light Sky Blue
    'hardware_app_version': PatternFill(start_color="98FB98", end_color="98FB98", fill_type="solid")  # Pale Green
}

# 应用填充样式
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=final_result.shape[1]):
    field_value = row[-1].value
    if field_value in fill_patterns:
        fill = fill_patterns[field_value]
        for cell in row:
            cell.fill = fill

# 为‘封号比例’列添加数据条格式
data_bar_rule = DataBarRule(start_type="num", start_value=0, end_type="num", end_value=1, color="FF6347", showValue="None")
col_index = final_result.columns.get_loc('封号比例') + 1
ws.conditional_formatting.add(f'{chr(65+col_index)}2:{chr(65+col_index)}{ws.max_row}', data_bar_rule)

# 将‘封号比例’列格式修改为保留两位小数的百分比格式
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=col_index, max_col=col_index):
    for cell in row:
        cell.number_format = '0.00%'

# 保存 Excel 文件前，检查文件是否存在
if os.path.exists(excel_file):
    # 如果文件存在，先删除原文件
    os.remove(excel_file)

# 保存 Excel 文件，此时原文件已被删除，所以不会报错
wb.save(excel_file)

print(f"Excel file '{excel_file}' has been created with conditional formatting.")


Excel file 'C:\Users\Administrator\Desktop\tg注册封号分析\result2.xlsx' has been created with conditional formatting.
